In [ ]:
from skimage.measure import shannon_entropy
import flirimageextractor
import cv2 as cv
import numpy as np

In [ ]:
img1 = cv.imread('./Assets/samples/Visual/1.jpg') 
img2 = cv.imread('./Assets/samples/Visual/2.jpg') 
img3 = cv.imread('./Assets/samples/Visual/3.jpg')
img4 = cv.imread('./Assets/samples/Visual/4.jpg')
img5 = cv.imread('./Assets/samples/Visual/5.jpg') 
img6 = cv.imread('./Assets/samples/Visual/6.jpg') 
img7 = cv.imread('./Assets/samples/Visual/7.jpg')
img8 = cv.imread('./Assets/samples/Visual/8.jpg')
img9 = cv.imread('./Assets/samples/Visual/9.jpg')

# thermal
Timg2 = cv.imread('./Assets/samples/Thermal/2.jpg') 
Timg1 = cv.imread('./Assets/samples/Thermal/1.jpg') 
Timg3 = cv.imread('./Assets/samples/Thermal/3.jpg')
Timg4 = cv.imread('./Assets/samples/Thermal/4.jpg')
Timg5 = cv.imread('./Assets/samples/Thermal/5.jpg') 
Timg6 = cv.imread('./Assets/samples/Thermal/6.jpg') 
Timg7 = cv.imread('./Assets/samples/Thermal/7.jpg')
Timg8 = cv.imread('./Assets/samples/Thermal/8.jpg')
Timg9 = cv.imread('./Assets/samples/Thermal/9.jpg')

print(img1.shape, Timg1.shape)

In [ ]:
# alpha represents weight of first image

def hsvFusion(image1, image2, alpha):

    tmp1 = cv.cvtColor(image1, cv.COLOR_BGR2GRAY, dstCn = 1)
    tmp2 = cv.cvtColor(image2, cv.COLOR_BGR2GRAY, dstCn = 1)

    hsv_image = cv.cvtColor(image2, cv.COLOR_BGR2HSV)
    hsv_channels = cv.split(hsv_image)

    res_channel = cv.addWeighted(tmp1, alpha, tmp2, 1-alpha, 0.0)

    dst = cv.merge([hsv_channels[0], hsv_channels[1], res_channel])
    dst = cv.cvtColor(dst, cv.COLOR_HSV2BGR)

    return dst
   # cv.imwrite('saved.jpg', dst)

In [ ]:
def genFocus(image, vis, thres1, thres2):
    edges1 = cv.Canny(image, thres1, thres2)
    edges2 = cv.Canny(vis, thres1, thres2)
    copy = image.copy()
    
    cv.imshow('dst', edges1)
    cv.waitKey(0)
    cv.destroyAllWindows()  
    
    contours1, _ = cv.findContours(edges1, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
    contours2, _ = cv.findContours(edges2, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
    
    cnt1 = max(contours1, key = cv.contourArea)
    cnt2 = max(contours2, key = cv.contourArea)
    
    coord1 = cv.boundingRect(cnt1)
    coord2 = cv.boundingRect(cnt2)

    x,y,w,h = max([coord1, coord2], key = lambda k : k[2]*k[3])
    print(x,y,w,h)
    t = cv.rectangle(copy, (x, y), (x+w, y+h), (0, 255, 0), 1)  
    
    cv.imshow('dst', t)
    cv.waitKey(0)
    cv.destroyAllWindows()

    print(w,h)
    return x, y, w, h

In [ ]:
def focus(image, vis, sz, K, iter, vals):

    if(iter == 0):
        return genFocus(image, vis, 60, 100)
    
    if(sz[1] != 0):
        temp = image[:, sz[0]:sz[1]]
        vis_temp = vis[:, sz[0]:sz[1]]
    else:
        temp = image.copy()
        vis_temp = vis.copy()
    
    tmp = np.float32(temp.reshape((-1, 3)))
    
    criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 10, 1.0)

    _, label, center = cv.kmeans(tmp, K, None, criteria, 10, cv.KMEANS_RANDOM_CENTERS)


    center = np.uint8(center)
    res = center[label.flatten()].reshape((temp.shape))

    cv.imwrite('color_map.jpg', res)
    cv.imshow('res', res)
    cv.waitKey(0)
    cv.destroyAllWindows()

    base_X, base_Y, ext_X, ext_Y = colorSeg(res, sz[0], vals[iter-1])

    t = temp[base_Y:base_Y + ext_Y, base_X:base_X + ext_X]
    v = vis_temp[base_Y:base_Y + ext_Y, base_X:base_X + ext_X].astype(np.uint8)
    x, y, w, h = focus(t, v, [0,0], 4, iter-1, vals)

    return base_X + x, base_Y + y, w, h

In [ ]:
def colorSeg(image, base, vals):
    image_hsv = cv.cvtColor(image, cv.COLOR_BGR2HSV)
    h, _, _ = cv.split(image_hsv)
    bins = np.bincount(h.flatten())

    peaks = np.where(bins > 0)[0]
    print(peaks)

    m,n = vals
    pres = False
    for i in peaks:
        if i>=m and i<=n:
            pres = True
            break
    
    print(pres)
    if pres:
        mask = cv.inRange(h, m, n)
    else:
        mask = cv.inRange(h, int(peaks[0]), int(peaks[0]))

    blob = cv.bitwise_and(image, image, mask=mask)

    cv.imshow('res', blob)
    cv.waitKey(0)
    cv.destroyAllWindows()

    contours, _ = cv.findContours(mask, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

    cnt = max(contours, key=cv.contourArea)
    x,y,w,h = cv.boundingRect(cnt)
    print(x,y,w,h)
    copy = image.copy()

    cv.rectangle(copy, (x,y), (x+w, y+h), (0, 255, 0), 1)

    cv.imshow('res', copy)
    cv.waitKey(0)
    cv.destroyAllWindows()

    return x, y, w, h

In [ ]:
def ENT(coord, fused):
    x1, y1, x2, y2 = coord
    
    print(shannon_entropy(fused[y1:y2, x1:x2]))

def thermal_mean(coord, path):
    x1, y1, x2, y2 = coord

    flir = flirimageextractor.FlirImageExtractor()
    flir.process_image(path)

    print(np.mean(flir.get_thermal_np()[y1:y2, x1:x2]))

In [ ]:
def maskROI(coord, image):
    x1,y1,x2,y2 = coord
    mask = np.zeros(image.shape[:2], dtype="uint8")
    cv.rectangle(mask, (x1,y1), (x2,y2), 255, -1)

    masked = cv.bitwise_and(image, image, mask=mask)
    cv.imshow('res', masked)
    cv.waitKey(0)
    cv.destroyAllWindows()

    return (x2-x1)*(y2-y1)

In [ ]:
def readImg(image1, image2, path):

    # set function here
    dst = hsvFusion(image1, image2, 0.7)
    dst = cv.resize(dst, (image1.shape[1], image1.shape[0]))
    cv.imwrite('fused.jpg', dst)
    
    #define parameters
    vals = [ (10, 10), (5, 15), (0, 20) ]
    r = [80,250]
    
    x,y,w,h = focus(dst, image1, r, 6, 3, vals)
    x1, y1, x2, y2 = x+r[0], y, x+r[0]+w, y+h
    coords = (x1,y1,x2,y2)
    
    copy = dst.copy()
    cv.rectangle(copy, (x1, y1), (x2, y2), (0, 255, 0), 1)
    cv.imwrite('saved.jpg', copy)
    
    roi = cv.imread('saved.jpg')
    fused = cv.imread('fused.jpg')

    cv.imshow('res', roi)
    cv.waitKey(0)
    cv.destroyAllWindows()

    ENT(coords, fused)
    thermal_mean(coords, path)
    area = maskROI(coords, fused)
    print(area)

    return np.concatenate((image1, image2, fused), axis=1), fused

dst, fused = readImg(img4, Timg4, './Assets/samples/Thermal/4.jpg')
cv.imshow('dst', dst)
cv.waitKey(0)
cv.destroyAllWindows()